In [ ]:
# coding: UTF-8
import sys
import os
import requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd

In [ ]:
USER_ID = ""

In [ ]:
URL = f"https://kenkoooo.com/atcoder/atcoder-api/results?user={USER_ID}" # アクセスするURL

Valid_max_point = 2000 # 考慮するポイントの最大値 マラソン 系を除くため
Today = datetime.today()

# 全期間
Total_problems = 0
Total_score = 0
Latest_AC_Date = datetime.date(Today - timedelta(days=1000))

# 2ヵ月
Two_month_before = datetime.date(Today - relativedelta(months = 2))
Latest_two_month_score = 0

# 1週間
One_week_before = datetime.date(Today - timedelta(days=7))
Weekly_AC_Days = 0
Weekly_AC_problems = 0

In [ ]:
ac_data = requests.get(URL)

In [ ]:
ac_list = eval(ac_data.text)

In [ ]:
submit_df = pd.DataFrame(ac_list)

In [ ]:
submit_df

In [ ]:
    submit_df = pd.DataFrame(ac_list)
    ac_df = submit_df.query('result == "AC" & point <= @Valid_max_point').copy()
    ac_df['date'] = ac_df['epoch_second'].apply(lambda x: datetime.date(datetime.fromtimestamp(x)))
    ac_df['score'] = ac_df['point']**2
    ac_df = ac_df[['date', 'problem_id', 'point', 'score']]

In [ ]:
    # 全期間の合計
    global Total_problems, Total_score, Latest_AC_Date
    Total_problems = ac_df.count()['problem_id']
    Total_score = ac_df.sum()['score']
    Latest_AC_Date = ac_df.sort_values('date', ascending=False)['date'][0]

In [ ]:
    # 最近2ヶ月のkpiグラフ
    global Latest_two_month_score
    Latest_two_month_score = ac_df.query('date >= @Two_month_before').sum()['score']
    hoge = ac_df.query('date >= @Two_month_before').groupby('date').sum()
    ax = hoge['point'].plot(marker='o', figsize=(12, 8), xlim=[Two_month_before, Today], grid=True, title='AtCoder point and score')
    hoge['score'].plot(ax=ax, marker='o', figsize=(12, 8), xlim=[Two_month_before, Today], grid=True, title='AtCoder point and score')
    #plt.savefig('point_score.png')

In [ ]:
    # 最近1週間
    global Weekly_AC_Days, Weekly_AC_problems
    Weekly_AC_Days = ac_df.query('date >= @One_week_before')['date'].nunique()
    Weekly_AC_problems = ac_df.query('date >= @One_week_before').count()['problem_id']

In [ ]:
        message_all = "AtCoderで解いた問題数: " + str(Total_problems) + ", pointの2乗和: "+ str(Total_score) + ", 直近のAC: " + str(Latest_AC_Date) + "\n"
        message_2month = "直近2ヵ月のAC スコア合計: " + str(Latest_two_month_score) + "\n"
        message_1week = "直近1週間のAC: " + str(Weekly_AC_Days) + "日間で"+ str(Weekly_AC_problems) + "問\n"
        message = message_all + message_2month + message_1week

In [ ]:
print(message)